In [1]:
import os
import sys
sys.version

#sys.path.append('/anaconda3/envs/tensorflow/lib/python3.6/site-packages/')

'3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) \n[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]'

In [2]:
import tensorflow as tf
import numpy as np
#import pandas as pd

In [3]:
# h = [1,0,0,0]
# e = [0,1,0,0]
# l = [0,0,1,0]
# 0 = [0,0,0,1]
_onehot = np.array([[[1,0,0,0]],[[0,1,0,0]],[[0,0,1,0]],[[0,0,1,0]]],dtype="float32")
onehot = np.array([[[0,1,0,0]],[[0,0,1,0]],[[0,0,1,0]],[[0,0,0,1]]],dtype="float32")

In [4]:
#weight and bias
with tf.variable_scope('weight') as scope:
    weight = tf.Variable(tf.random_normal(shape=[3,2,4,4]),dtype="float32")
with tf.variable_scope('bias') as scope:
    bias = tf.Variable(tf.random_normal(shape=[3,1,4]),dtype="float32")
#accumulate gradiant value
with tf.variable_scope('acc_weight') as scope:
    acc_weight = tf.Variable(tf.zeros(shape=[3,2,4,4]),dtype="float32")
with tf.variable_scope('acc_bias') as scope:
    acc_bias = tf.Variable(tf.zeros(shape=[3,1,4]),dtype="float32")

In [5]:
weight

<tf.Variable 'weight/Variable:0' shape=(3, 2, 4, 4) dtype=float32_ref>

In [6]:
with tf.variable_scope('input') as scope:
    inputv = tf.placeholder(dtype="float32",shape=[1,4])
with tf.variable_scope('label') as scope:
    labelv = tf.placeholder(dtype="float32",shape=[1,4])

In [7]:
with tf.variable_scope('hid') as scope:
    hiddenv = tf.Variable([[0,0,0,0]],dtype="float32")
    _hiddenv = tf.Variable([[0,0,0,0]],dtype="float32")
    last_hiddenv = tf.Variable([[0,0,0,0]],dtype="float32")

In [8]:
with tf.variable_scope('z') as scope:
    z_pre = tf.add(tf.matmul(inputv,weight[0][0]), tf.matmul(hiddenv,weight[0][1]))
    z_pres = tf.add(z_pre,bias[0])
    z = tf.sigmoid(z_pres)
    
    inputv_hist = tf.summary.histogram("inputv_hist",inputv)
    hiddenv_hist = tf.summary.histogram("hiddenv_hist",hiddenv)
    weight_00_hist = tf.summary.histogram("weight_00_hist",weight[0][0])
    weight_01_hist = tf.summary.histogram("weight_01_hist",weight[0][1])
    z_hist = tf.summary.histogram("z_hist",z)


In [9]:
z

<tf.Tensor 'z/Sigmoid:0' shape=(1, 4) dtype=float32>

In [10]:
with tf.variable_scope('r') as scope:
    r_pre = tf.add(tf.matmul(inputv,weight[1][0]), tf.matmul(hiddenv,weight[1][1]))
    r_pres = tf.add(r_pre,bias[1])
    r = tf.sigmoid(r_pres)
    
    inputv_hist2 = tf.summary.histogram("inputv_hist",inputv)
    hiddenv_hist2 = tf.summary.histogram("hiddenv_hist",hiddenv)
    weight_10_hist = tf.summary.histogram("weight_00_hist",weight[1][0])
    weight_11_hist = tf.summary.histogram("weight_01_hist",weight[1][1])
    r_hist = tf.summary.histogram("r_hist",r)

In [11]:
r

<tf.Tensor 'r/Sigmoid:0' shape=(1, 4) dtype=float32>

In [12]:
with tf.variable_scope('hidden_') as scope:
    _hiddenv_pre = tf.add(tf.matmul(inputv,weight[2][0]),tf.matmul(hiddenv,weight[2][1])*r)
    _hiddenv_pres = tf.add(bias[2],_hiddenv_pre)
    _hiddenv = tf.tanh(_hiddenv_pres)
    
    inputv_hist3 = tf.summary.histogram("inputv_hist",inputv)
    hiddenv_hist3 = tf.summary.histogram("hiddenv_hist",hiddenv)
    r_hist2 = tf.summary.histogram("r_hist",r)
    weight_20_hist = tf.summary.histogram("weight_00_hist",weight[2][0])
    weight_21_hist = tf.summary.histogram("weight_01_hist",weight[2][1])
    hiddenv_ = tf.summary.histogram("hiddenv_",_hiddenv)

In [13]:
with tf.variable_scope('last_hid') as scope:
    f_lhiddenv = z*hiddenv
    s_lhiddenv = (1-z)*_hiddenv
    last_hiddenv = f_lhiddenv + s_lhiddenv

    z_hid = tf.summary.histogram("z",z)
    f_lhiddenv_hist = tf.summary.histogram("f1",f_lhiddenv)
    s_lhiddenv_hist = tf.summary.histogram("s1",s_lhiddenv)
    
# hidden state를 다음 time step에 넘겨줌
with tf.variable_scope('to_next') as scope:
    as_hid = tf.assign(hiddenv,last_hiddenv)

In [14]:
# cost와 gradient를 계산
with tf.variable_scope('cost') as scope:
    cost = tf.square(last_hiddenv - labelv)
    cost_hid = tf.summary.histogram("cost",cost)
    dw = tf.gradients(cost,weight)
    db = tf.gradients(cost,bias)

In [15]:
with tf.variable_scope('bptt') as scope:
    #weight bias 의 gradiant누적
    as_acc = tf.assign(acc_weight,tf.add(acc_weight,dw)[0])# [0]은 차원을 맞추기위한것, 아무의미없음
    as_accb = tf.assign(acc_bias,tf.add(acc_bias,db)[0])
    #weight 갱신
    asw = tf.assign(weight,tf.subtract(weight,acc_weight))
    #hiddenenv 초기화
    ash = tf.assign(hiddenv,tf.Variable([[0,0,0,0]],dtype="float32"))
    #bias 갱신
    asb = tf.assign(bias,tf.subtract(bias,acc_bias))
    # acc값 초기화
    re_accw = tf.assign(acc_weight,tf.zeros(shape=[3,2,4,4]))
    re_accb = tf.assign(acc_bias,tf.zeros(shape=[3,1,4]))

In [16]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./gru")
    writer.add_graph(session.graph)
    i = 0
    step = 0
    while (i < 1000):
        
        while (step < 4):
            session.run([z_pre],feed_dict={inputv:_onehot[step]})
            session.run([z_pres],feed_dict={inputv:_onehot[step]})
            session.run([z],feed_dict={inputv:_onehot[step]})
            
            session.run([r_pre],feed_dict={inputv:_onehot[step]})
            session.run([r_pres],feed_dict={inputv:_onehot[step]})
            session.run([r],feed_dict={inputv:_onehot[step]})
            
            session.run([_hiddenv_pre],feed_dict={inputv:_onehot[step]})
            session.run([_hiddenv_pres],feed_dict={inputv:_onehot[step]})
            session.run([_hiddenv],feed_dict={inputv:_onehot[step]})
            
            session.run([f_lhiddenv],feed_dict={inputv:_onehot[step]})
            session.run([s_lhiddenv],feed_dict={inputv:_onehot[step]})
            session.run([last_hiddenv],feed_dict={inputv:_onehot[step]})
            
            print("\n cost = ",session.run([cost],feed_dict={inputv:_onehot[step], labelv:onehot[step]}))
            session.run([dw,db],feed_dict={inputv:_onehot[step], labelv:onehot[step]})
            session.run([as_acc,as_accb],feed_dict={inputv:_onehot[step], labelv:onehot[step]})
            session.run([as_hid],feed_dict={inputv:_onehot[step], labelv:onehot[step]}) #다음 step으로 hidden값을 넘겨줌
            print("step = ",step)
            step = step + 1
        step = 0
        
       
    
        session.run([asw,asb])
        session.run([ash],feed_dict={inputv:_onehot[step], labelv:onehot[step]})
        session.run([re_accw],feed_dict={inputv:_onehot[step], labelv:onehot[step]})
        session.run([re_accb],feed_dict={inputv:_onehot[step], labelv:onehot[step]})
        
        s = session.run(summary,feed_dict={inputv:_onehot[step], labelv:onehot[step]})
        writer.add_summary(s,i)
        i = i + 1




 cost =  [array([[ 0.60098845,  0.12038191,  0.02258577,  0.20864175]], dtype=float32)]
step =  0

 cost =  [array([[ 0.60300511,  0.74632549,  0.43897721,  0.08341549]], dtype=float32)]
step =  1

 cost =  [array([[ 0.70805931,  0.4685559 ,  0.3665058 ,  0.55454183]], dtype=float32)]
step =  2

 cost =  [array([[ 0.85436916,  0.33683261,  0.21438821,  0.14745331]], dtype=float32)]
step =  3

 cost =  [array([[  3.47308457e-01,   4.96839523e-01,   1.77222057e-04,
          3.82260352e-01]], dtype=float32)]
step =  0

 cost =  [array([[ 0.36159891,  0.16117665,  0.16364557,  0.54333121]], dtype=float32)]
step =  1

 cost =  [array([[ 0.54289711,  0.22040071,  0.05579812,  0.37180659]], dtype=float32)]
step =  2

 cost =  [array([[ 0.55378467,  0.26613408,  0.72565013,  0.145202  ]], dtype=float32)]
step =  3

 cost =  [array([[  2.09795564e-01,   2.25608665e-02,   7.32032231e-06,
          6.76381350e-01]], dtype=float32)]
step =  0

 cost =  [array([[ 0.19190088,  0.97425854,  0.75087